In [ ]:
def make_ims_2d(im_marker, im_dapi):
    '''Takes in two images in 3d and compresses them into 2D image. Takes in 
    two numpy arrays and returns two numpy arrays.'''
    labeled_im_marker = skimage.measure.label(im_marker)
    labeled_im_dapi = skimage.measure.label(im_dapi)
    
    # Making marker im 2D
    for region in skimage.measure.regionprops(labeled_im_marker):
        for coord in region.coords:
            (x,y,z) = coord
            labeled_im_marker[x][y][0] = 0
            labeled_im_marker[x][y][1] = 1
            labeled_im_marker[x][y][2] = 0
            
    for region in skimage.measure.regionprops(labeled_im_dapi):
        for coord in region.coords:
            (x,y,z) = coord
            labeled_im_dapi[x][y][0] = 1
            labeled_im_dapi[x][y][1] = 0
            labeled_im_dapi[x][y][2] = 0
    return (labeled_im_marker, labeled_im_dapi)

In [ ]:
def apply_dapi_segmentation(im_marker, im_dapi):
    '''Takes an input of an image array as a numpy array segmented in the 
    DAPI channel and a numpy array segmented in another channel,
    both already labeled and compressed to 2D array and applies the DAPI 
    segmentation to the image. 
    
    Returns an image array as a numpy array.'''
    (labeled_im_marker, labeled_im_dapi) = make_ims_2d(im_marker, im_dapi)
            
    # Initialize dictionary for number of markers in cells
    num_markers_in_cell = {}
    
            
    # Make DAPI 2D and add Xist onto layer of RGB image
    for region in skimage.measure.regionprops(labeled_im_dapi):
        for coord in region.coords:
            (x, y, z) = coord
            labeled_im_dapi[x][y][0] = 1
            labeled_im_dapi[x][y][2] = 0
            if labeled_im_marker[x][y][1] == 1:
                labeled_im_dapi[x][y][1] = 1
    return labeled_im_dapi

In [ ]:
def marker_areas(im_marker, im_dapi):
    '''Takes in an input of an image with the dapi and marker channels merged.
    
    Creates a distribution of the areas of the markers within a cell and only takes
    the brightest % based on the dist_thresh value'''
    
    (labeled_im_marker, labeled_im_dapi) = make_ims_2d(im_marker, im_dapi)
    merged_im = apply_dapi_segmentation(labeled_im_marker, labeled_im_dapi)
    area_dct = {}
    area_lst = []
    
    for dapi_region in skimage.measure.regionprops(labeled_im_dapi):
        dapi_coords = dapi_region.coords
        (x_dapi, y_dapi, z_dapi) = dapi_coords
        for marker_region in skimage.measure.regionprops(labeled_im_marker):
            marker_coords = marker_region.coords
            (x_marker, y_marker, z_marker) = marker_coords
            i = 0
            for marker_coord in marker_coords:
                if labeled_im_marker[x_marker][y_marker][1] == 1:
                    if marker_coord in dapi_coords:
                        for i in range(1):
                            area = marker_region.area
                            centroid = marker_region.centroid
                            area_dct[centroid] = area
                            area_lst.append(area)
                            i += 1
    area_array = np.array(area_lst)
    return area_dct, area_array